In [1]:
import pymongo
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

In [2]:
client = MongoClient()
db = client.companies
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### Based on the information provided, we are going to select a possible location for the office based on the available information.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company ***expect to increase their employees number in the future***, so being continously changing the location would be counterproductive.

Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [4]:
pd.DataFrame.from_dict(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [5]:
filt2 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project2 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results2 = db.offices.find(filt2,project2).sort("number_of_employees",1)#.skip(1).limit(1)
results2

In [6]:
office_location = list(results2)
office_location

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

In [7]:
pd.DataFrame.from_dict(office_location)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


## OFFICE LOCATION

In [8]:
office_lat = 40.7592189
office_long = -73.9783534

In [9]:
office_data = [("office","40.7592189","-73.9783534","POINT (-73.9783534 40.7592189)","office location")]
df_office = pd.DataFrame(office_data, columns = ["name","latitud","longitud","geometry","criteria"]) 
df_office

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location


In [10]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 16)
map_1

In [11]:
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [12]:
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [13]:
type(office_location)

list

In [14]:
load_dotenv()

True

In [130]:
city = "New York"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [131]:
donde = "New York"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [132]:
print(data)

{'standard': {'addresst': {}, 'city': 'New York', 'prov': 'US', 'countryname': 'United States of America', 'postal': {}, 'confidence': '0.90'}, 'longt': '-73.95861', 'alt': {'loc': {'longt': '-73.9536869565217', 'prov': 'NY', 'city': 'NEW YORK', 'postal': '10075', 'score': '25', 'latt': '40.7724386956522'}}, 'elevation': {}, 'latt': '40.68908'}


In [129]:
nyc = geocode(city)

In [19]:
nyc

{'type': 'Point', 'coordinates': [-73.95861, 40.68908]}

In [20]:
nyc = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}

In [21]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Búsqueda de Starbucks

In [22]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [23]:
parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500
}

In [24]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)

In [25]:
data.keys()

dict_keys(['meta', 'response'])

In [26]:
decoding_data = data.get("response")

In [27]:
decoded = decoding_data.get("groups")[0]

In [28]:
starbucks = decoded.get("items")

In [113]:
map_starbucks = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [114]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [115]:
print(getFromDict(starbucks[0],map_starbucks))

Starbucks


In [116]:
starbucks_nyc = []
for dic in starbucks:
    lis1 = {}
    lis1["name"] = getFromDict(dic,map_starbucks)
    lis1["latitud"] = getFromDict(dic,m_latitude)
    lis1["longitud"] = getFromDict(dic,m_longitude)
    starbucks_nyc.append(lis1)

In [117]:
starbucks_nyc[0]

{'name': 'Starbucks',
 'latitud': 40.75814852475969,
 'longitud': -73.97941981951507}

In [34]:
df_starbucks = pd.DataFrame(starbucks_nyc)
df_starbucks.head()

,name,latitud,longitud
0,Starbucks,40.758149,-73.979420
1,Starbucks,40.759535,-73.980981
2,Starbucks,40.760602,-73.976467
3,Starbucks,40.761845,-73.978700
4,Starbucks,40.756665,-73.974397


### OJO
**20 Starbucks** en un radio de 500m de la oficina

In [35]:
len(df_starbucks.name)

20

In [36]:
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud,df_starbucks.latitud))
gdf_starbucks.head()

,name,latitud,longitud,geometry
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815)
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953)
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060)
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185)
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666)


In [37]:
gdf_starbucks = gdf_starbucks.assign(name = [ "Starbucks"]*20,
               criteria = ["Starbucks"]*20)
gdf_starbucks.head()

,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185),Starbucks
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666),Starbucks


In [38]:
Map(Layer(gdf_starbucks, popup_hover = [popup_element("name","Starbucks in NYC")]))

### 2. Looking for vegan restaurants

In [39]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [40]:
parametros_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500
}

In [41]:
resp2 = requests.get(url = url_query, params = parametros_vegan)
data2 = json.loads(resp2.text)

In [42]:
data2.keys()

dict_keys(['meta', 'response'])

In [43]:
decoding_data2 = data2.get("response")

In [44]:
decoded2 = decoding_data2.get("groups")[0]

In [45]:
vegan = decoded2.get("items")

In [46]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [47]:
def getFromDict(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [48]:
print(getFromDict(vegan[0],map_vegan))

Beyond Sushi


In [49]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [50]:
vegan_nyc[0]

{'name': 'Beyond Sushi',
 'latitud': 40.76345369164551,
 'longitud': -73.9771627466244}

In [51]:
df_vegans = pd.DataFrame(vegan_nyc)
df_vegans.head()

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


In [52]:
len(df_vegans.name)

30

### OJO
**30 restaurantes veganos** en un radio de 500m de la oficina

In [53]:
gdf_vegan = gpd.GeoDataFrame(df_vegans, geometry = gpd.points_from_xy(df_vegans.longitud,df_vegans.latitud))
gdf_vegan.head()

,name,latitud,longitud,geometry
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345)
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806)
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958)
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029)
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876)


In [54]:
gdf_vegan = gdf_vegan.assign(name = ["Beyond Sushi",
                                        "by CHLOE.",
                                        "Van Leeuwen Ice Cream",
                                        "Taco Dumbo",
                                        "L’Avenue",
                                        "Taco Dumbo",
                                        "Urbanspace W52",
                                        "The Halal Guys",
                                        "The Little Beet",
                                        "Lenwich by Lenny's",
                                        "Le Pain Quotidien",
                                        "Mysttik Masala",
                                        "Museum of Modern Art (MoMA)",
                                        "Fogo de Chão",
                                        "Aldo Sohm Wine Bar",
                                        "Gregorys Coffee",
                                        "Pret A Manger",
                                        "Devon & Blakely",
                                        "Fig & Olive",
                                        "The Modern",
                                        "Black Tap",
                                        "Simon Sips",
                                        "Barilla Restaurants",
                                        "Estiatorio Milos",
                                        "Le Pain Quotidien",
                                        "Cock & Bull British Pub and Eatery",
                                        "Europa Cafe",
                                        "Forty2West",
                                        "Butter Midtown",
                                        "Natureworks"],
                                criteria = ["vegan"]*30)
gdf_vegan.head()

,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345),vegan
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806),vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958),vegan
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029),vegan
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876),vegan


In [55]:
Map(Layer(gdf_vegan, popup_hover = [popup_element("name","Vegan restaurants in NYC")]))

### 3. Looking for a basketball stadium


In [56]:
url3 = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"
# Ubicado menos a 10 km

In [57]:
parametros_basket = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000
}

In [58]:
resp3 = requests.get(url = url_query, params = parametros_basket)
data3 = json.loads(resp3.text)

In [59]:
data3.keys()

dict_keys(['meta', 'response'])

In [60]:
decoding_data3 = data3.get("response")

In [61]:
decoded3 = decoding_data3.get("groups")[0]

In [62]:
basket_stadium = decoded3.get("items")

In [63]:
map_stadium = ["venue","name"]

In [64]:
m_latitud_stadium = ["venue","location","lat"]
m_longitud_stadium = ["venue","location","lng"]
def getFromDict(diccionario3,mapa3):
    return reduce (operator.getitem,mapa3,diccionario3)

In [65]:
print(getFromDict(basket_stadium[0],map_stadium))

Regal E-Walk 4DX & RPX


In [66]:
basket_stadium_nyc = []
for dic in basket_stadium:
    paralista3 = {}
    paralista3["name"] = getFromDict(dic,map_stadium)
    paralista3["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista3["longitud"] = getFromDict(dic,m_longitud_stadium)
    basket_stadium_nyc.append(paralista3)

In [67]:
basket_stadium_nyc[0]
df_stadium = pd.DataFrame(basket_stadium_nyc)
df_stadium.head()

,name,latitud,longitud
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286
1,Zaro's Bakery,40.752875,-73.976946
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395


In [68]:
len(df_stadium)

4

### OJO
**4 estadios de baloncesto** een un radio de 1000m de la oficina. **22 estadios** si ampliamos el radio a 10 km.

In [69]:
basketball_stadiums_location = list(df_stadium)
basketball_stadiums_location

['name', 'latitud', 'longitud']

In [70]:
gdf_stadiums = gpd.GeoDataFrame(df_stadium, geometry = gpd.points_from_xy(df_stadium.longitud,df_stadium.latitud))
gdf_stadiums

,name,latitud,longitud,geometry
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696)
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288)
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784)
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721)


In [71]:
gdf_stadiums = gdf_stadiums.assign(name = [ "Regal E-Walk 4DX & RPX",
                           "Zaro's Bakery",
                           "Boomer Esiason's Stadium Grill",
                           "Stadium Grill At Bowlmor Lanes"],
               criteria = ["basketball stadium","basketball stadium","basketball stadium","basketball stadium"])
gdf_stadiums.head()

,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696),basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288),basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784),basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721),basketball stadium


In [72]:
Map(Layer(gdf_stadiums, popup_hover = [popup_element("name","Basketball Stadium")]))

### 4. Night clubs

Nightlife Spot = "4d4b7105d754a06376d8125"

In [73]:
url4 = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d11f941735"

In [74]:
parametros_night_clubs = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"night_clubs","radius":1000
}

resp4 = requests.get(url = url_query, params = parametros_night_clubs)
data4 = json.loads(resp4.text)

In [75]:
data4.keys()

dict_keys(['meta', 'response'])

In [118]:
decoding_data4 = data4.get("response")
decoded4 = decoding_data4.get("groups")[0]
night_clubs = decoded4.get("items")

In [79]:
map_night_clubs = ["venue","name"]

In [80]:
m_latitud_nightclubs = ["venue","location","lat"]
m_longitud_nightclubs = ["venue","location","lng"]

In [81]:
def getFromDict(diccionario4,mapa4):
    return reduce (operator.getitem,mapa4,diccionario4)

In [82]:
print(getFromDict(night_clubs[0],map_night_clubs))

Lavo


In [83]:
night_clubs_nyc = []
for dic in night_clubs:
    paralista4 = {}
    paralista4["name"] = getFromDict(dic,map_night_clubs)
    paralista4["latitud"] = getFromDict(dic,m_latitud_nightclubs)
    paralista4["longitud"] = getFromDict(dic,m_longitud_nightclubs)
    night_clubs_nyc.append(paralista4)

In [84]:
night_clubs_nyc[0]
df_night_clubs = pd.DataFrame(night_clubs_nyc)
df_night_clubs.head()

,name,latitud,longitud
0,Lavo,40.762919,-73.971338
1,Studio 54,40.763944,-73.983980
2,JOE & THE JUICE,40.765877,-73.981975
3,Tao,40.762659,-73.971441
4,STK Steakhouse Midtown NYC,40.755262,-73.982888


In [85]:
len(df_night_clubs)

11

### OJO

**1 clubs nocturnos** a un radio de 1000m de la oficina.

In [86]:
night_clubs_location = list(df_night_clubs)
night_clubs_location

['name', 'latitud', 'longitud']

In [87]:
gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs.head()

,name,latitud,longitud,geometry
0,Lavo,40.762919,-73.971338,POINT (-73.97134 40.76292)
1,Studio 54,40.763944,-73.983980,POINT (-73.98398 40.76394)
2,JOE & THE JUICE,40.765877,-73.981975,POINT (-73.98197 40.76588)
3,Tao,40.762659,-73.971441,POINT (-73.97144 40.76266)
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,POINT (-73.98289 40.75526)


In [89]:
gdf_night_clubs = gdf_night_clubs.assign(name = ["Lavo",
                                        "Studio 54",
                                        "JOE & THE JUICE",
                                        "Tao",
                                        "STK Steakhouse Midtown NYC",
                                        "The Smith",
                                        "The View Restaurant & Lounge",
                                        "Chivas 1801 Club- NYC",
                                        "Flowers by Richard",
                                        "The Mean Fiddler",
                                        "Public House"],
                    criteria = ["night_clubs"]*11)
gdf_night_clubs.head()

,name,latitud,longitud,geometry,criteria
0,Lavo,40.762919,-73.971338,POINT (-73.97134 40.76292),night_clubs
1,Studio 54,40.763944,-73.983980,POINT (-73.98398 40.76394),night_clubs
2,JOE & THE JUICE,40.765877,-73.981975,POINT (-73.98197 40.76588),night_clubs
3,Tao,40.762659,-73.971441,POINT (-73.97144 40.76266),night_clubs
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,POINT (-73.98289 40.75526),night_clubs


In [90]:
Map(Layer(gdf_night_clubs, popup_hover = [popup_element("name","Night clubs")]))

### 5. Transport availability

In [91]:
url5 = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [92]:
parametros_travel = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"travel_transport",
                ## "radius":1000
                    }

In [93]:
resp5 = requests.get(url = url_query, params = parametros_travel)
data5 = json.loads(resp5.text)

In [94]:
data5.keys()

dict_keys(['meta', 'response'])

In [119]:
decoding_data5 = data5.get("response")
decoded5 = decoding_data5.get("groups")[0]
transport = decoded5.get("items")

In [120]:
map_transport = ["venue","name"]

In [121]:
m_latitud_transport = ["venue","location","lat"]
m_longitud_transport = ["venue","location","lng"]

In [122]:
def getFromDict(diccionario5,mapa5):
    return reduce (operator.getitem,mapa5,diccionario5)

In [99]:
print(getFromDict(transport[0],map_transport))

Theater Row - The Acorn


In [100]:
transport_nyc = []
for dic in transport:
    paralista5 = {}
    paralista5["name"] = getFromDict(dic,map_stadium)
    paralista5["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista5["longitud"] = getFromDict(dic,m_longitud_stadium)
    transport_nyc.append(paralista5)

In [101]:
transport_nyc[0]

{'name': 'Theater Row - The Acorn',
 'latitud': 40.7585514688034,
 'longitud': -73.9934189068339}

In [102]:
df_transports = pd.DataFrame(transport_nyc)
df_transports.head()

,name,latitud,longitud
0,Theater Row - The Acorn,40.758551,-73.993419
1,Solstice Travel Vacations,40.758305,-73.967880
2,The Travel Inn,40.760124,-73.996154
3,United Bus and Travel,40.713707,-73.991714
4,Double Happiness Travel,40.713707,-73.991714


In [103]:
len(df_transports)

9

### OJO

**9 puntos de transporte** cercanos a la oficina para que los KAM puedan viajar fácilmente.

In [104]:
gdf_transports = gpd.GeoDataFrame(df_transports, geometry = gpd.points_from_xy(df_transports.longitud,df_transports.latitud))
gdf_transports.head()

,name,latitud,longitud,geometry
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855)
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830)
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012)
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371)
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371)


In [123]:
gdf_transports = gdf_transports.assign(name = [ "Theater Row - The Acorn",
                           "Solstice Travel Vacations",
                           "The Travel Inn",
                           "United Bus and Travel",
                           "Double Happiness Travel",
                           "K International Transport Co., Inc",
                           "Delgado Travel",
                           "Active Transport ServicesInc.",
                           "Rivas Travel & Multi Services"],
               criteria = ["transports"]*9)
gdf_transports.head()

,name,latitud,longitud,geometry,criteria
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855),transports
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830),transports
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012),transports
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports


In [106]:
Map(Layer(gdf_transports, popup_hover = [popup_element("name","Transports")]))

## Finally, we concatenate our criterias' dataframes.

In [107]:
df_criterias = pd.concat([df_office,gdf_starbucks, gdf_vegan,gdf_stadiums,gdf_night_clubs,gdf_transports])
df_criterias.head()

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location
0,Starbucks,40.758149,-73.979420,POINT (-73.97941981951507 40.75814852475969),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098084615312 40.75953488433791),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97646651001223 40.76060249450284),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.9787 40.761845),Starbucks


In [108]:
df_criterias = df_criterias

In [109]:
df_criterias.criteria.unique()

array(['office location', 'Starbucks', 'vegan', 'basketball stadium',
       'night_clubs', 'transports'], dtype=object)

In [110]:
map_f = folium.Map (location = [40.7592189,-73.9783534],zoom_start = 15, min_lon = - 180, max_lon = 180, attr = 'Mapbox attribution')

In [111]:
for i,row in df_criterias.iterrows():
    name = {"location" : [row["latitud"],row["longitud"]],
            "tooltip" : row["name"]}
    
    if row["criteria"] == 'Starbucks':
        icon = Icon(color = "darkpurple",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "beige")
    elif row["criteria"] == 'transports':
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "globe",
                    icon_color = "beige")
    elif row["criteria"] == 'office location':
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "building-o",
                    icon_color = "beige")
    elif row["criteria"] == 'basketball stadium':
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "beige")
    elif row["criteria"] == 'vegan':
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "cutlery",
                    icon_color = "beige")
    else:
        icon = Icon(color = "white",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "lightblue")
        
    Marker(**name,icon = icon).add_to(map_f)
    
map_f

In [112]:
map_f.save("mapa.html")